In [ ]:
import sys
from pathlib import Path
sys.path.append(
    str(Path.cwd().parent.resolve())
)

In [ ]:
import argparse
import subprocess
from multiprocessing import cpu_count
from typing import List, Set

import pandas as pd
import numpy as np

from rdkit import RDLogger
from rdkit.Chem import rdChemReactions
from rdkit.DataStructs.cDataStructs import ConvertToNumpyArray

from openTSNE import TSNE

import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns

import src.utils as ut
from src import root
from src.prediction import MTReagentPredictor, MTProductPredictor
from src.preprocessing import HeuristicRoleClassifier, SOLVENTS

In [ ]:
from IPython.display import display

In [ ]:
RDLogger.DisableLog('rdApp.*')

In [ ]:
def remove_redundant_separators(smi: str) -> str:
    """
    Removes trailing or repeated separators from a SMILES string
    :param smi: A SMILES string encoding several molecules
    :return: A SMILES string with no redundant separators
    """
    return ".".join([i for i in smi.split('.') if i != ''])


def standardize_pd_pph3(smi: str) -> str:
    """
    Replaces a Pd(PPh3)4 molecule with 5 separate species - one Pd and four PPh3
    :param smi:
    :return:
    """
    united = "c1ccc([PH](c2ccccc2)(c2ccccc2)[Pd]([PH](c2ccccc2)(c2ccccc2)c2ccccc2)([PH](c2ccccc2)(c2ccccc2)c2ccccc2)[PH](c2ccccc2)(c2ccccc2)c2ccccc2)cc1"
    split = "[Pd].c1ccc(P(c2ccccc2)c2ccccc2)cc1.c1ccc(P(c2ccccc2)c2ccccc2)cc1.c1ccc(P(c2ccccc2)c2ccccc2)cc1.c1ccc(P(c2ccccc2)c2ccccc2)cc1"
    return smi.replace(united, split)


def match_accuracy(tgt_and_pred: 'pd.Series', mode: str) -> List[int]:
    """
    Calculates the accuracy of reagent predictions.
    :param tgt_and_pred: The series, where the first row containts the target string,
    and all the other contain predictions ordered by priority.
    The function is meant to be applied to a DataFrame of shape (M, N+1), where M is the number of reactions
    and N is the number of predictions.
    :param mode: If it is 'exact', calculates exact match accuracy.
    If it is 'partial', calculates partial match accuracy.
    :return: List of zeros and ones of length N.
    """

    def _exact_match(t_set: Set[str], p_set: Set[str]) -> bool:
        return len(t_set.symmetric_difference(p_set)) == 0

    def _partial_match(t_set: Set[str], p_set: Set[str]) -> bool:
        return len(t_set & p_set) > 0

    if mode == 'exact':
        _match = _exact_match
    elif mode == 'partial':
        _match = _partial_match
    else:
        raise ValueError(f"The only allowed values for the 'mode' argument are 'exact' and 'partial', not {mode}")

    tgt_mols, *pred_mols = tgt_and_pred
    tgt_set = {s for s in tgt_mols.split(".") if s}
    topn = {i + 1: False for i in range(len(pred_mols))}
    for i, p in enumerate(pred_mols):
        pred_set = {s for s in p.split(".") if s}
        topn[i + 1] = topn.get(i, False) or _match(tgt_set, pred_set)
    return [int(topn[i + 1]) for i in range(len(pred_mols))]


def recall(tgt_and_pred: 'pd.Series') -> List[float]:
    """
    Calculates the recall of reagent predictions.
    :param tgt_and_pred: The series, where the first row containts the target string,
    and all the other contain predictions ordered by priority.
    The function is meant to be applied to a DataFrame of shape (M, N+1), where M is the number of reactions
    and N is the number of predictions.
    :param mode: If it is 'exact', calculates exact match accuracy.
    If it is 'partial', calculates partial match accuracy.
    :return: List of recall scores of length N.
    """

    tgt_mols, *pred_mols = tgt_and_pred
    tgt_set = {s for s in tgt_mols.split(".") if s}
    tgt_set_orig_len = len(tgt_set)
    topn = {i + 1: 0 for i in range(len(pred_mols))}
    for i, p in enumerate(pred_mols):
        pred_set = {s for s in p.split(".") if s}
        tgt_set = tgt_set - pred_set
        topn[i + 1] = 1 - len(tgt_set) / tgt_set_orig_len
    return [topn[i + 1] for i in range(len(pred_mols))]


def plot_confidence_distributions(confs: 'pd.Series', match: 'pd.Series'):
    fig, ax = plt.subplots(ncols=2, nrows=1, sharey=True, figsize=(10, 10))

    plt.sca(ax[0])
    sns.violinplot(y=confs,
                   scale="width",
                   ax=plt.gca())
    plt.grid(axis='y')

    plt.sca(ax[1])
    sns.boxplot(x=match, y=confs, ax=plt.gca())
    plt.grid(axis='y')

    return fig, ax


def plot_performance_across_classes(classes_col: 'pd.Series',
                                    exact_accuracies: 'pd.DataFrame',
                                    partial_accuracies: 'pd.DataFrame',
                                    width: float = 0.2):
    """
    Plots the performance of reagent prediction across reaction classes.
    :param classes_col: The column with a reaction class name for each reaction
    :param exact_accuracies: The dataframe with exact match accuracies.
    Columns are thought to be ordered by priority in descending order.
    :param partial_accuracies: The dataframe with partial match accuracies.
    Columns are thought to be ordered by priority in descending order.
    :param width: The width of bars to plot
    :return:
    """
    fig, ax = plt.subplots(nrows=2, ncols=1, sharex=True, sharey=False, figsize=(24, 12))

    colors_exact = {1: "#239b56", 3: "#58d68d", 5: "#abebc6"}
    colors_partial = {1: "#922b21", 3: "#cd6155", 5: "#e6b0aa"}

    ascending_top_1_order = classes_col.value_counts().index
    _accs_classes = pd.concat((classes_col, exact_accuracies, partial_accuracies), axis=1)
    class_col = _accs_classes.columns[0]
    for i in (5, 3, 1):
        score_col = exact_accuracies.columns[i - 1]
        ax[0].bar(
            np.arange(len(ascending_top_1_order)),
            _accs_classes.groupby(by=[class_col])[score_col].mean()[ascending_top_1_order],
            width,
            color=colors_exact[i],
            alpha=1,
            label=f"Top-{i} exact"
        )
        score_col = partial_accuracies.columns[i - 1]
        ax[0].bar(
            np.arange(len(ascending_top_1_order)) + width,
            _accs_classes.groupby(by=[class_col])[score_col].mean()[ascending_top_1_order],
            width,
            color=colors_partial[i],
            alpha=1,
            label=f"Top-{i} partial"
        )

    ax[0].tick_params(length=0)
    plt.sca(ax[0])
    plt.legend()
    plt.grid(axis='y', alpha=0.5)
    plt.title(
        "Fraction of partial and perfect matches between the target sequence and the predicted sequence across reaction classes.",
        fontsize=16)

    ax[1].bar(
        np.arange(len(ascending_top_1_order)) + width / 2,
        _accs_classes[class_col].value_counts(normalize=True)[ascending_top_1_order],
        width,
        color='b',
        alpha=0.6,
    )

    ax[0].set_yticks(np.arange(0.0, 1.0, 0.1))

    ax[1].tick_params(length=0)
    plt.sca(ax[1])
    plt.xticks(np.arange(len(ascending_top_1_order)) + width / 2, ascending_top_1_order, rotation=19, fontsize=14)

    # plt.xticks(rotation=15)
    plt.xlabel(None)
    plt.grid(axis='y', alpha=0.5)
    plt.title("Distribution of reaction classes in the test.", fontsize=16)

    fig.suptitle(
        f"Performance of the transformer model on the test set.\nTest size: {test.shape[0]} reactions.",
        fontsize=16)

In [ ]:
CLASS_COLORS = {
    "Acylation and related processes": "#C0392B",
    "Heteroatom alkylation and arylation": "#E67E22",
    "C-C bond formation": "#27AE60",
    "Heterocycle formation": "#F1C40F",
    "Protections": "#1186F3",
    "Deprotections": "#707B7C",
    "Reductions": "#40C4DE",
    "Oxidations": "#DC40DE",
    "Functional group interconversion (FGI)": "#CCAF1C",
    "Functional group addition (FGA)": "#8E44AD"
}


class ReactionFPS:
    """
    Calculates reaction fingerprints using RDKit.
    """

    FP_TYPES = {"AtomPairFP": rdChemReactions.FingerprintType.AtomPairFP,
                "MorganFP": rdChemReactions.FingerprintType.MorganFP,
                "PatternFP": rdChemReactions.FingerprintType.PatternFP,
                "RDKitFP": rdChemReactions.FingerprintType.RDKitFP,
                "TopologicalTorsion": rdChemReactions.FingerprintType.TopologicalTorsion
                }

    def calculate(self,
                  rx_smi: str,
                  fp_method: str,
                  n_bits: int,
                  fp_type: str,
                  include_agents: bool,
                  agent_weight: int,
                  non_agent_weight: int,
                  bit_ratio_agents: float = 0.2
                  ) -> 'np.array':
        """
        Calculates reaction fingerprints for a given reaction SMILES string.
        More info on arguments: https://www.rdkit.org/docs/cppapi/structRDKit_1_1ReactionFingerprintParams.html
        :param rx_smi: Reaction SMILES to calculate fingerprints for.
        :param fp_method: 'structural' or 'difference'.
        :param n_bits: Number of bits in the fingerprint vectors
        :param fp_type: the algorithm for fingerprints, e.g. AtompairFP.
        Be aware that only AtompairFP, TopologicalTorsion and MorganFP are supported in the difference fingerprint.
        :param include_agents: a flag: include the agents of a reaction for fingerprint generation or not
        :param agent_weight: if agents are included, agents could
        be weighted compared to reactants and products in difference fingerprints.
        :param non_agent_weight: in difference fingerprints weight factor for reactants and products compared to agents
        :param bit_ratio_agents: in structural fingerprints it determines the ratio of bits of the agents in the fingerprint
        :return: fingerprint vector (numpy array)
        """
        # === Parameters section
        params = rdChemReactions.ReactionFingerprintParams()
        params.fpSize = n_bits
        params.includeAgents = include_agents
        params.fpType = self.FP_TYPES[fp_type]
        # ===

        rxn = rdChemReactions.ReactionFromSmarts(
            rx_smi,
            useSmiles=True)

        arr = np.zeros((1,))
        if fp_method == "difference":
            params.agentWeight = agent_weight
            params.nonAgentWeight = non_agent_weight
            # NOTE: difference fingerprints are not binary
            fps = rdChemReactions.CreateDifferenceFingerprintForReaction(rxn, params)

        elif fp_method == "structural":
            params.bitRatioAgents = bit_ratio_agents
            # NOTE: structural fingerprints are binary
            fps = rdChemReactions.CreateStructuralFingerprintForReaction(rxn, params)
        else:
            raise ValueError("Invalid fp_method. Allowed are 'difference' and 'structural'")

        ConvertToNumpyArray(fps, arr)
        return arr


def diff_fp(smi: str) -> 'np.array':
    return ReactionFPS().calculate(smi,
                                   fp_method="difference",
                                   n_bits=2048,
                                   fp_type="MorganFP",
                                   include_agents=True,
                                   agent_weight=1,
                                   non_agent_weight=1)


def plot_2d_distribution(x,
                         y,
                         save_path=None,
                         ax=None,
                         title=None,
                         draw_legend=True,
                         colors=None,
                         legend_kwargs=None,
                         label_order=None,
                         **kwargs) -> None:
    """
    Plots TSNE embeddings
    """
    if ax is None:
        _, ax = plt.subplots(figsize=(20, 20))

    if title is not None:
        ax.set_title(title)

    plot_params = {"alpha": kwargs.get("alpha", 0.6), "s": kwargs.get("s", 1)}

    # Create main plot
    if label_order is not None:
        assert all(np.isin(np.unique(y), label_order))
        classes = [l for l in label_order if l in np.unique(y)]
    else:
        classes = np.unique(y)
    if colors is None:
        default_colors = matplotlib.rcParams["axes.prop_cycle"]
        colors = {k: v["color"] for k, v in zip(classes, default_colors())}

    point_colors = list(map(colors.get, y))

    ax.scatter(x[:, 0], x[:, 1], c=point_colors, rasterized=True, **plot_params)

    # Hide ticks and axis
    ax.set_xticks([]), ax.set_yticks([]), ax.axis("off")

    if draw_legend:
        legend_handles = [
            matplotlib.lines.Line2D(
                [],
                [],
                marker="s",
                color="w",
                markerfacecolor=colors[yi],
                ms=10,
                alpha=1,
                linewidth=0,
                label=yi,
                markeredgecolor="k",
            )
            for yi in classes
        ]
        legend_kwargs_ = dict(loc="best", bbox_to_anchor=(1, 0.5), frameon=False, )
        if legend_kwargs is not None:
            legend_kwargs_.update(legend_kwargs)
        ax.legend(handles=legend_handles, **legend_kwargs_)

    if save_path is not None:
        plt.savefig(save_path, dpi=300)
    else:
        plt.show()


def plot_rxn_class_proportions(classes_in: 'pd.Series',
                               classes_out: 'pd.Series',
                               save_path=None) -> None:
    """
    Plot the graph comparing the proportions of the ten reaction classes from USPTO 50K
    between the in-distribution data and the out-of-distribution data.
    :param save_path: Path to save the image.
    :param classes_in: Proportions of reaction classes for the in-distribution data.
    :param classes_out: Proportions of reaction classes for the out-of-distribution data.
    :return:
    """

    plt.figure(figsize=(10, 5))
    width = 0.3

    plt.bar(
        np.arange(len(classes_in)),
        classes_in,
        width,
        color='#abebc6',
        alpha=1,
        label="USPTO 50K"
    )
    plt.bar(
        np.arange(len(classes_in)) + width,
        classes_out[classes_in.index],
        width,
        color='#cd6155',
        alpha=1,
        label="Reaxys Test"
    )
    plt.legend()
    plt.xticks(np.arange(len(classes_in)) + width / 2,
               classes_in.index,
               rotation=75,
               fontsize=11)
    plt.title("Proportion of reaction classes")
    plt.grid(axis='y')

    plt.tight_layout()
    if save_path is not None:
        plt.savefig(save_path, dpi=300)
    else:
        plt.show()

In [ ]:
np.random.seed(123456)

### Arguments

In [ ]:
USPTO50_PATH = "../uspto50k.csv"

TEST_PATH = "../data/raw/reaxys_test_full_nice_classes.csv"
VOCAB = "../data/vocabs/reag_no_MIT_src_vocab.json"
MODEL = "../experiments/trained_models/reag_no_MIT_model_step_80000_shared_voc.pt"
# MODEL = "../experiments/trained_models/simple_tok_with_rep_aug_model_step_520000.pt"
RUN_NAME = "reaxys_rgs_80k_test_full"
SRC_COLUMN = "ReactionRgsSource"
GPU = 0

# Loading the test set and USPTO 50K for visual comparison

In [ ]:
# Data which is in-distribution to the training set
data_in = pd.read_csv("../uspto50k.csv")

# Test data
data_out = pd.read_csv(TEST_PATH).fillna('')

In [ ]:
stat_in = data_in["General class"].value_counts(normalize=True)

stat_out = data_out["General class"].value_counts(normalize=True)

In [ ]:
stat_in

In [ ]:
stat_out

### Comparing reaction class proportions

In [ ]:
plot_rxn_class_proportions(stat_in, stat_out)

### Comparing the TSNE embeddings of poins in USPTO50K and the test set

In [ ]:
tsne = TSNE(
        perplexity=30,
        metric="euclidean",
        n_jobs=cpu_count(),
        random_state=123456,
        verbose=True
    )

In [ ]:
fp_in = np.vstack(
        data_in["Reaction"].apply(diff_fp)
    )
embedding_in = tsne.fit(fp_in)

TSNE is parametric here. It produces embeddings for the test reactions based on the embeddings of reactions in USPTO 50K.

In [ ]:
fp_out = np.vstack(
        data_out["FullR"].apply(diff_fp)
    )
embedding_out = embedding_in.transform(fp_out)

In [ ]:
plot_2d_distribution(embedding_in,
                     data_in["General class"],
                     colors=CLASS_COLORS,
                     s=10, 
                     title="TSNE map of USPTO 50K")

In [ ]:
plot_2d_distribution(embedding_out,
                     data_out["General class"],
                     s=10,
                     title="TSNE map of the test set",
                     colors={i: CLASS_COLORS.get(i, "#00FF00") for i in data_out["General class"].unique()})

In [ ]:
plot_2d_distribution(
        np.vstack((embedding_in, embedding_out)),
        np.array(["USPTO 50K"] * embedding_in.shape[0] + ["Reaxys Test"] * embedding_out.shape[0]),
        title="TSNE maps of both USPTO 50K and the test set in overlap"
    )

In [ ]:
for c in CLASS_COLORS:
    if c in data_in["General class"].unique() and c in data_out["General class"].unique():
        _u = embedding_in[data_in["General class"] == c]
        _r = embedding_out[data_out["General class"] == c]
        both = np.vstack((_u, _r))
        h = sns.jointplot(x=both[:, 0],
                          y=both[:, 1],
                          kind='scatter',
                          alpha=0.7,
                          palette=["#000000", CLASS_COLORS[c]],
                          hue=np.array(["USPTO 50K"] * _u.shape[0] + ["Reaxys Test"] * _r.shape[0]))
        h.set_axis_labels("TSNE 1", "TSNE 2")
        plt.suptitle(f"TSNE embeddings of points in USPTO 50K and the test set.\nClass: {c}")
        plt.tight_layout()

# Loading the test set for reagent prediction

In [ ]:
PREDICT = True

In [ ]:
test = pd.read_csv(TEST_PATH).fillna('')

In [ ]:
test.head()

In [ ]:
test.shape

In [ ]:
test["General class"].value_counts(normalize=True)

# Reagent prediction model

The transformer predicts the reagents for a reaction in the form of a SMILES string. We trained our models to predict the reagents for the broad range of reactions without restrictions: the number of reagents and their particular roles are not predetermined. The model yields the output sequentially, token by token, treating the choice of a token as a multiclass classification problem and conditioning that choice on the input sequence and the previously decoded tokens. As we arranged the molecules in the target sequences in the training set by their roles in a reaction, the model is trained to first predict metal catalysts conditioned on the input sequence, if they are necessary, then redox agents conditioning on the input sequence and predicted catalysts, then acids and bases, then uncategorized molecules and ions like alkali metal cations, and finally solvents conditioned on everything mentioned. The solvents are the most interchangeable and they require the most context to predict. A similar reagent role ordering was also used in Gao et al. \cite{Gao2018}.

In [ ]:
reag_predictor = MTReagentPredictor(
        vocabulary_path=VOCAB,
        model_path=MODEL,
        tokenized_path=str((root / "data" / "test" / ("src_" + RUN_NAME)).with_suffix(".txt")),
        output_path=str((root / "experiments" / "results" / RUN_NAME).with_suffix(".txt")),
        beam_size=5,
        n_best=5,
        batch_size=32,
        gpu=GPU
    )

In [ ]:
%%time
if PREDICT:
    reag_predictor.predict(test[SRC_COLUMN])

### Loading predictions

In [ ]:
reag_predictor.load_predictions()
pred_rgs = reag_predictor.predictions
pred_rgs_conf = reag_predictor.pred_probs

### Cleaning up predictions a bit

In [ ]:
for c in pred_rgs.columns:
    pred_rgs[c] = pred_rgs[c].apply(remove_redundant_separators)
    pred_rgs[c] = pred_rgs[c].apply(ut.IonAssembler.run)
    pred_rgs[c] = pred_rgs[c].apply(standardize_pd_pph3)

### Printing the number of predicted SMILES which are invalid

The model captures the SMILES language structure nicely - ***practically*** all of the predictions of reagent models are valid SMILES strings. 

In [ ]:
for c in pred_rgs.columns:
    print(c, (pred_rgs[c] == '').sum())

# Calculating accuracy

We analyze the model's performance from several different perspectives:  
  * Perfect match accuracy:   
    The proportion of the predicted SMILES strings in which the molecules exactly match the groung truth molecules.
  * Patial match accuracy:   
    The proportion of the predicted SMILES strings in which at least one of the molecules is in the groung truth set. 
  * Precision (proportion of the predicted molecules present in the GT sequence) and recall (coverage of the ground truth sequence):   
    **Not implemented - requires a higher overall score**

In [ ]:
rgs_and_predictions = pd.concat(
        (test["Reagents"],
         pred_rgs),
        axis=1
    )
rgs_and_predictions.head()

### Exact match accuracy

In [ ]:
topn_exact_match_acc = rgs_and_predictions.apply(
        lambda x: match_accuracy(x, 'exact'),
        axis=1
    )
topn_exact_match_acc = pd.DataFrame(topn_exact_match_acc.to_list())
topn_exact_match_acc.columns = [f"top_{i + 1}_exact" for i in range(pred_rgs.shape[1])]

print("Top-N exact match accuracy")
for c in topn_exact_match_acc.columns:
    print(c, topn_exact_match_acc[c].mean())

### Partial match accuracy

In [ ]:
topn_partial_match_acc = rgs_and_predictions.apply(
        lambda x: match_accuracy(x, 'partial'),
        axis=1
    )
topn_partial_match_acc = pd.DataFrame(topn_partial_match_acc.to_list())
topn_partial_match_acc.columns = [f"top_{i + 1}_partial" for i in range(pred_rgs.shape[1])]

print("Top-N partial match accuracy")
for c in topn_partial_match_acc.columns:
    print(c, topn_partial_match_acc[c].mean())

### Recall

In [ ]:
topn_recall = rgs_and_predictions.apply(
        lambda x: recall(x),
        axis=1
    )
topn_recall = pd.DataFrame(topn_recall.to_list())
topn_recall.columns = [f"top_{i + 1}_recall" for i in range(pred_rgs.shape[1])]

In [ ]:
topn_recall

In [ ]:
plt.plot(
    topn_recall["top_5_recall"].value_counts().sort_index(),
    '--o',
);

In [ ]:
for c in topn_recall.columns:
    print(c, (topn_recall[c] == 1).mean())

### Accuracy without solvents

In [ ]:
rgs_and_predictions

In [ ]:
def remove_solvents(smi: str) -> str:
    """
    Removes standard solvents from a sequence of molecules separated by dots 
    """
    return '.'.join([i for i in smi.split('.') if i not in SOLVENTS])

In [ ]:
rgs_and_predictions_no_solvents = pd.DataFrame()
for c in rgs_and_predictions.columns:
    rgs_and_predictions_no_solvents[c] = rgs_and_predictions[c].apply(remove_solvents)

In [ ]:
rgs_and_predictions_no_solvents.head()

In [ ]:
topn_exact_match_acc_no_solvents = rgs_and_predictions_no_solvents.apply(
        lambda x: match_accuracy(x, 'exact'),
        axis=1
    )
topn_exact_match_acc_no_solvents = pd.DataFrame(topn_exact_match_acc_no_solvents.to_list())
topn_exact_match_acc_no_solvents.columns = [f"top_{i + 1}_exact" for i in range(pred_rgs.shape[1])]

In [ ]:
print("Top-N exact match accuracy")
for c in topn_exact_match_acc_no_solvents.columns:
    print(c, topn_exact_match_acc_no_solvents[c].mean())

In [ ]:
topn_partial_match_acc_no_solvents = rgs_and_predictions_no_solvents.apply(
        lambda x: match_accuracy(x, 'partial'),
        axis=1
    )
topn_partial_match_acc_no_solvents = pd.DataFrame(topn_partial_match_acc_no_solvents.to_list())
topn_partial_match_acc_no_solvents.columns = [f"top_{i + 1}_partial" for i in range(pred_rgs.shape[1])]
print("Top-N partial match accuracy")
for c in topn_partial_match_acc_no_solvents.columns:
    print(c, topn_partial_match_acc_no_solvents[c].mean())

# How many unique reactions have at least one reagent set predicted right?

In [ ]:
# Number of unique reactions in the test set
test["ReactionRgsSource"].nunique()

In [ ]:
# How many duplicates does every unique reaction have
test["ReactionRgsSource"].value_counts().value_counts().sort_index()

In [ ]:
unique_rxn_perf = pd.concat(
    (
        test["ReactionRgsSource"], 
        topn_exact_match_acc,
        topn_partial_match_acc
    ), 
    axis=1
).groupby(by="ReactionRgsSource")

In [ ]:
unique_rxn_perf["top_1_exact"].max()

In [ ]:
for c in topn_exact_match_acc.columns:
    print(c, unique_rxn_perf[c].max().mean())
print()  
for c in topn_partial_match_acc.columns:
    print(c, unique_rxn_perf[c].max().mean())

# Confidence of predictions

In [ ]:
fig, ax = plot_confidence_distributions(
    pred_rgs_conf["p_reagents_1_conf"],
    topn_exact_match_acc["top_1_exact"]
)
fig.suptitle("Distribution of the confidence scores of the model")
ax[0].set_title("Overall distribution of confidence scores")
ax[1].set_title("Confidence scores for reactions\n predicted correctly and not")
ax[1].set_xlabel("Top-1 exact match")
ax[0].set_ylabel("Confidence of the first prediction")
ax[1].set_ylabel(None)
ax[0].set_xlabel("Proportion of reactions")
plt.sca(ax[1])
plt.show()

# Performance across reaction classes

In [ ]:
plot_performance_across_classes(
    test["General class"], 
    topn_exact_match_acc, 
    topn_partial_match_acc
)
plt.show()

Correlates with the TSNE maps similarity of corresponding reaction classes

# Performance across reagent roles

### Roles of the target reagents

In [ ]:
test_r_roles = pd.concat(
        (
            test["General class"],
            test["Reagents"].apply(HeuristicRoleClassifier.classify_to_str).str.split("&", expand=True)
        ),
        axis=1
    )
test_r_roles.columns = ["General class"] + [f"tgt_{r}" for r in HeuristicRoleClassifier.types]
test_r_roles.head()

In [ ]:
test_r_roles.iloc[10000:10005]

### Roles of the predicted reagents for each of the N predictions

In [ ]:
n = pred_rgs.shape[1]
top_n_roles = {}
for i in range(1, n + 1):
    _temp = pred_rgs[f"p_reagents_{i}"].apply(
        HeuristicRoleClassifier.classify_to_str
    ).str.split("&",
                expand=True)
    _temp.columns = [f"pred_{r}" for r in HeuristicRoleClassifier.types]
    top_n_roles[i] = _temp

In [ ]:
for i in top_n_roles:
    print(f"Pred {i}")
    display(top_n_roles[i].head())

### Top-N exact match of predictions for each role

In [ ]:
role_performance = {}
for r in HeuristicRoleClassifier.types:
    
    tgt_and_pred_roles = pd.concat(
        [test_r_roles[f"tgt_{r}"]] + [top_n_roles[i][f"pred_{r}"] for i in range(1, n + 1)],
        axis=1
    )
    perf = tgt_and_pred_roles.apply(
        lambda x: match_accuracy(x, 'exact'),
        axis=1
    )
    perf = pd.DataFrame(perf.to_list())
    perf.columns = [f"top_{i}_exact" for i in range(1, n + 1)]
    role_performance[r] = perf

In [ ]:
role_perf_summary = {}
for r in HeuristicRoleClassifier.types:
    nonempty_target = test_r_roles[test_r_roles[f"tgt_{r}"] != ''].index
    role_perf_summary[r] = {
        "Top-1": role_performance[r]["top_1_exact"].mean(),
        "Top-1 non-empty GT": role_performance[r].loc[nonempty_target]["top_1_exact"].mean(),
        "Top-5": role_performance[r]["top_5_exact"].mean(),
        "Top-5 non-empty GT": role_performance[r].loc[nonempty_target]["top_5_exact"].mean()
    }

In [ ]:
role_perf_summary

In [ ]:
role_perf_summary = pd.DataFrame.from_dict(role_perf_summary)
role_perf_summary

In [ ]:
role_perf_summary = role_perf_summary.T
role_perf_summary

In [ ]:
plt.figure(figsize=(8, 8))
sns.heatmap(role_perf_summary, annot=True, cmap="mako", ax=plt.gca())
plt.title(
    "Proportion of test examples on which the prediction\nmatches the ground truth exactly in each reagent role.",
    fontsize=14)
plt.yticks(rotation=0, fontsize=14)
plt.xticks(fontsize=13)
# plt.xticks(fontsize=16)
plt.tight_layout()
plt.show()

# Performance on reaction classes vs reagent roles

In [ ]:
role_top_1_match_by_class = []
role_top_1_match_by_class_nonempty = []
role_nonempty_count = []

for r in HeuristicRoleClassifier.types:
    
    # Stacking the groung truth reagents in one role against prediction performance on that role
    _temp = pd.concat(
        (
            test_r_roles[["General class", f"tgt_{r}"]], 
            role_performance[r]
        ),
        axis=1
    )
    # Need to be careful with empty GT
    _temp["tgt_nonempty"] = _temp[f"tgt_{r}"].apply(lambda x: x != '')
    
    # Proportion of matches by class
    role_top_1_match_by_class.append(
        _temp.groupby(by="General class")["top_1_exact"].mean()
    )
    
    # Proportion of matches by class for nonempty GT
    role_top_1_match_by_class_nonempty.append(
        _temp[_temp["tgt_nonempty"]].groupby(by="General class")["top_1_exact"].mean()
    )
    
    # Number of nonempty GT per class
    role_nonempty_count.append(
        _temp.groupby(by="General class")["tgt_nonempty"].sum()
    )

# Concatenating information for each role    

role_top_1_match_by_class = pd.concat(role_top_1_match_by_class, axis=1)
role_top_1_match_by_class.columns = HeuristicRoleClassifier.types

role_top_1_match_by_class_nonempty = pd.concat(role_top_1_match_by_class_nonempty, axis=1)
role_top_1_match_by_class_nonempty.columns = HeuristicRoleClassifier.types

role_nonempty_count = pd.concat(role_nonempty_count, axis=1)
role_nonempty_count.columns = HeuristicRoleClassifier.types

In [ ]:
fig, ax = plt.subplots(ncols=3, nrows=1, sharey=True, figsize=(30, 12))
angle = 45

sns.heatmap(role_top_1_match_by_class, 
            annot=True, 
            ax=ax[0], 
            fmt='.3g',
            cmap="mako")
plt.sca(ax[0])
plt.ylabel(None)
plt.xticks(rotation=angle, fontsize=14)
plt.title("Top-1 accuracy of predictions by roles across classes")
plt.yticks(fontsize=9)

sns.heatmap(role_top_1_match_by_class_nonempty, 
            annot=True, 
            ax=ax[1], 
            fmt='.3g',
            cmap="mako")
plt.sca(ax[1])
plt.ylabel(None)
plt.xticks(rotation=angle, fontsize=14)
plt.title("Top-1 accuracy of predictions by roles across classes\nfor nonempty GT")
plt.yticks(fontsize=9)

sns.heatmap(role_nonempty_count, 
            annot=True, 
            fmt='d',
            ax=ax[2], 
            cmap="mako")
plt.sca(ax[2])
plt.title("Number of non-empty GT")
plt.ylabel(None)
plt.xticks(rotation=angle, fontsize=14)
plt.yticks(fontsize=9)

fig.suptitle("Performance of reagent prediction across reaction classes and reagent roles", fontsize=16)
plt.show()

# Comparing the number of predicted molecules with the length of GT

In [ ]:
n_tgt_mols = test["Reagents"].apply(lambda x: len(x.split('.')))

In [ ]:
def label_difference(n: int):
    if n == 0:
        return "Same"
    elif n < 0:
        return "Less"
    else:
        return "More"

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=pred_rgs.shape[1], sharey=True, figsize=(14, 4))
order = ["Less", "Same", "More"]
for i in range(pred_rgs.shape[1]):
    n_pred_mols = pred_rgs[f"p_reagents_{i + 1}"].apply(lambda x: len(x.split('.')))
    (n_pred_mols - n_tgt_mols).apply(label_difference).value_counts(normalize=True).loc[order].plot(kind='bar', ax=ax[i])
    ax[i].set_title(f"Prediction {i + 1}")

fig.suptitle("Number of predicted molecules compared to the number of molecules in GT")
plt.show()

The model tends to predict less molecules than needed

# Unique reagents in test

In [ ]:
unique_reagents = ut.get_reagent_statistics(test["Reagents"])

In [ ]:
print("Unique reagents in the test set")
len(unique_reagents)

In [ ]:
print("Unique reagents if they are repeated more than 20 times")
len([(k, v) for k, v in unique_reagents.items() if v >= 20])

# Performance across target lengths

### Across the number of molecules in target

In [ ]:
topn_exact_match_acc

In [ ]:
pd.concat(
    (test["Reagents"].apply(lambda x: len(x.split('.'))),
     topn_exact_match_acc),
     axis=1
).groupby(by="Reagents")["top_1_exact"].mean().plot(kind='bar')

In [ ]:
pd.concat(
    (test["Reagents"].apply(lambda x: len(x.split('.'))),
     topn_partial_match_acc),
     axis=1
).groupby(by="Reagents")["top_1_partial"].mean().plot(kind='bar')

### Across the target string length

In [ ]:
plt.figure(figsize=(13, 5))

plt.plot(
    pd.concat(
        (test["Reagents"].apply(len),
         topn_exact_match_acc),
         axis=1
    ).groupby(by="Reagents")["top_1_exact"].mean(),
    '-x'
)

plt.show()

In [ ]:
plt.figure(figsize=(13, 5))

plt.plot(
    pd.concat(
        (test["Reagents"].apply(len),
         topn_partial_match_acc),
         axis=1
    ).groupby(by="Reagents")["top_1_partial"].mean(),
    '-x'
)

plt.show()

Small improvement, like 3 percent points. Measuring performance like that improves the score, more duplicates needed.

# Number of unique predictions and unique GT sequence for each reaction class

In [ ]:
compar = pd.concat((test["General class"], rgs_and_predictions), axis=1)

In [ ]:
compar[compar["General class"] == "Reductions"]["p_reagents_1"].nunique()

In [ ]:
pd.concat(
    [compar.groupby(by="General class")[c].nunique() for c in compar.columns[1:]],
    axis=1
).style.background_gradient(cmap='Blues', axis=1)

Model's predictions are not particularly diverse

# Examples of incorrect predictions

Here the difference in train and test distributions very much manifests itself

In [ ]:
failed_in_exact = test.loc[topn_exact_match_acc[topn_exact_match_acc["top_5_exact"] == 0].index]
failed_in_partial = test.loc[topn_partial_match_acc[topn_partial_match_acc["top_5_partial"] == 0].index]

In [ ]:
c = "C-C bond formation"

In [ ]:
fail = failed_in_partial[failed_in_partial["General class"] == c]
fail_pred = pred_rgs.loc[fail.index]

In [ ]:
ix = iter(fail.index)

In [ ]:
i = next(ix)
print(i)
print("GT:", fail.loc[i]["Reagents"])
display(ut.draw_reaction_smarts(fail.loc[i]["FullR"]))
left, right = fail.loc[i]["ReactionRgsSource"].split(">>")
center = fail_pred.loc[i]["p_reagents_1"]
pr = ">".join((left, center, right))
print("Prediction:", center)
display(ut.draw_reaction_smarts(pr))

### Reactions with zero top-5 recall

In [ ]:
no_recall_index = topn_recall[topn_recall["top_5_recall"] == 0.0].index

In [ ]:
no_recall = test.loc[no_recall_index]
no_recall_pred = pred_rgs.loc[no_recall_index]

In [ ]:
test["General class"].value_counts()

In [ ]:
failing_classes = no_recall["General class"].value_counts().loc[test["General class"].value_counts().index] / test["General class"].value_counts()
failing_classes = failing_classes.sort_values()

In [ ]:
failing_classes

In [ ]:
plt.figure(figsize=(20, 5))
plt.bar(
    list(failing_classes.index),
    list(failing_classes)
)
plt.xticks(rotation=15, fontsize=11)
plt.grid(axis='y')
plt.title("Proportion of reactions in a class with zero top-5 recall within the test set")
plt.savefig("/home/mandronov/Pictures/no_recall.png")
plt.show()

In [ ]:
no_recall = no_recall[no_recall["General class"] == 'Functional group addition (FGA)']

In [ ]:
ix = iter(no_recall.index)

In [ ]:
i = next(ix)
print(i)
print("GT:", no_recall.loc[i]["Reagents"])
display(ut.draw_reaction_smarts(no_recall.loc[i]["FullR"]))
left, right = no_recall.loc[i]["ReactionRgsSource"].split(">>")
center = no_recall_pred.loc[i]["p_reagents_1"]
pr = ">".join((left, center, right))
print("Prediction:", center)
display(ut.draw_reaction_smarts(pr))

# Attention patterns

In [ ]:
att_sample_rxn = test.loc[14393]["ReactionRgsSource"]
att_sample_rxn

In [ ]:
sample_path_in = f"../data/test/att_sample.txt"
sample_path_out = f"../experiments/results/att_sample.txt"

In [ ]:
sample_pred = MTReagentPredictor(VOCAB, None, None, None)
tokenized_sample = " ".join(
    sample_pred.tokenizer_source.tokenize([att_sample_rxn])[0][1:-1]
)
with open(sample_path_in, "w") as f:
    f.write(tokenized_sample)

In [ ]:
tokenized_sample

In [ ]:
command = [
    "python3", "/home/mandronov/work/reagents/translate.py",
    "-model", MODEL,
    "-src", sample_path_in,
    "-output", sample_path_out,
    "-batch_size", "32",
    "-max_length", "200",
    "-beam_size", "5",
    "-n_best", "5", 
    "-attn_debug", "-replace_unk", "-fast",
    "-gpu", "0"]

att_w = subprocess.check_output(command)

In [ ]:
att_w = att_w.decode('utf-8').split("\n")

In [ ]:
source_tokens = att_w[0].split()

In [ ]:
lines = [i.split() for i in att_w[1:] if (i and not i.startswith("PRED"))]

output_tokens = []
scores = []
for line in lines:
    output_tokens.append(line[0])
    scores.append([float(j.strip('*')) for j in line[1:]])
scores = pd.DataFrame(scores)
scores.index = output_tokens
scores.columns = source_tokens

In [ ]:
plt.figure(figsize=(15, 10))
axx = plt.gca()
sns.heatmap(scores, cmap="rocket_r", ax=axx, xticklabels=True, yticklabels=True)
axx.tick_params(axis='both', which='both', length=0, labelsize=10, labelbottom=False, bottom=False, top=False, labeltop=True)

axx.set_xlabel("Source sequence", fontsize=16)
axx.set_ylabel("Output sequence", fontsize=16)
axx.xaxis.set_label_position('top') 
plt.xticks(rotation=0, fontsize=9)
plt.yticks(rotation=0, fontsize=9)
plt.show()

In [ ]:
left, right = "".join(source_tokens).split(">>")
decoded_reaction = left + ">" + "".join(output_tokens[:-1]) + ">" + right

In [ ]:
print(decoded_reaction)
ut.draw_reaction_smarts(decoded_reaction)

# Inspecting the replaced reagents in the MIT subset of USPTO

In [ ]:
path = "../data/tokenized/MIT_separated"
arrow = ">"

In [ ]:
subset = "train" # "val"

In [ ]:
with open(path + f"/src-{subset}.txt") as f, open(path + f"_no_reags/src-{subset}.txt") as g, open(path + f"_reags_top1/src-{subset}.txt") as h, open(path + f"_reags_top1_and_rdkit/src-{subset}.txt") as ff, open(path + f"_reags_role_voting/src-{subset}.txt") as gg:
    base = [i.strip().replace(" ", "") for i in f.readlines()]
    no_rgs = [i.strip().replace(" ", "") for i in g.readlines()]
    top = [i.strip().replace(" ", "") for i in h.readlines()]
    mix = [i.strip().replace(" ", "") for i in ff.readlines()]
    role = [i.strip().replace(" ", "") for i in gg.readlines()]
with open(path + f"/tgt-{subset}.txt") as f:
    tgt = [i.strip().replace(" ", "") for i in f.readlines()]

In [ ]:
base = [i + arrow + j for i, j in zip(base, tgt)]
no_rgs = [i + ">>" + j for i, j in zip(no_rgs, tgt)]
top = [i + arrow + j for i, j in zip(top, tgt)]
mix = [i + arrow + j for i, j in zip(mix, tgt)]
role = [i + arrow + j for i, j in zip(role, tgt)]

Examples of indexes of reactions with replaced reagents in train:  
1,   3,   6,   8,  17,  22,  25,  26,  33,  38,  39,  42,  47,
            51,  56,  59,  60,  64,  68,  76,  81,  83,  84,  87,  91,  92,
             94,  98,  99, 100, 102, 103, 105, 119, 121, 124, 129, 130, 133,
            136, 143, 144, 148, 151, 153, 154, 156, 162, 164, 168, 169, 178,
            180, 181, 182, 195, 200, 201, 203, 208, 209, 210, 211, 216, 222,
            228, 233, 243, 246, 249, 250, 254, 256, 257, 263, 266, 268, 273,
            274, 275, 277, 279, 284, 288, 289, 293, 294, 301, 309, 310, 321,
            324, 327, 328, 330, 332, 333, 334, 338, 342  
 
Examples of indexes of reactions with replaced reagents in val: 
0,   3,   5,   6,   8,  22,  24,  28,  35,  47,  48,  51,  53,
             59,  63,  67,  68,  72,  73,  76,  78,  79,  85,  88,  93,  97,
            106, 107, 108, 110, 118, 129, 134, 138, 140, 143, 145, 146, 148,
            156, 166, 169, 172, 173, 174, 186, 189, 194, 195, 199, 202, 209,
            211, 213, 214, 224, 226, 230, 231, 238, 243, 244, 245, 249, 250,
            251, 253, 262, 271, 272, 276, 277, 283, 290, 291, 292, 296, 302,
            309, 314, 316, 321, 327, 332, 334, 338, 342, 347, 351, 352, 364,
            372, 373, 377, 378, 390, 393, 395, 397, 405


In [ ]:
ix = iter(range(len(base)))

In [ ]:
i = next(ix)
print(i)
print("base")
display(ut.draw_reaction_smarts(base[i]))
print("mix")
display(ut.draw_reaction_smarts(mix[i]))
print("top")
display(ut.draw_reaction_smarts(top[i]))
print("no")
display(ut.draw_reaction_smarts(no_rgs[i]))
print("role")
display(ut.draw_reaction_smarts(role[i]))

# Products prediction

In [ ]:
# precursors = (test["Reactants"] + '.' + test["Reagents"]).str.strip('.')
precursors = test["Reactants"] + '>' + test["Reagents"]

In [ ]:
precursors

In [ ]:
precursors.nunique(), precursors.shape

In [ ]:
# MODELS = {
#     "mit_mixed_20k": "/home/mandronov/work/reagents/experiments/trained_models/MIT_mixed_augm_model_step_20000.pt",
#     "mit_mixed_40k": "/home/mandronov/work/reagents/experiments/trained_models/MIT_mixed_augm_model_step_40000.pt"
# }

MODELS = {
    "mit_sep_500k": "/home/mandronov/work/reagents/experiments/trained_models/MIT_separated_model_step_500000.pt",
    "mit_sep_500k_repl": "/home/mandronov/work/reagents/experiments/trained_models/MIT_separated_reags_top1_and_rdkit_model_step_500000.pt"
}

In [ ]:
product_predictors = {}
for name, model in MODELS.items():
    product_predictors[name] = MTProductPredictor(
        model_path=model,
        tokenized_path=f"../data/test/src_{name}.txt",
        output_path=f"../experiments/results/test_reaxys_{name}.txt",
        beam_size=5,
        n_best=5,
        gpu=0
    ) 

In [ ]:
product_predictors

In [ ]:
%%time
for k in product_predictors:
    product_predictors[k].predict(precursors)

In [ ]:
for k in product_predictors:
    product_predictors[k].load_predictions()

In [ ]:
for k in product_predictors:
    for c in product_predictors[k].predictions.columns:
        product_predictors[k].predictions[c] = product_predictors[k].predictions[c].apply(lambda smi: ut.canonicalize_smiles(max(smi.split('.'), key=len)))

In [ ]:
for k in product_predictors:
    hits = 0
    for i in range(test.shape[0]):
        for j in range(1, 5 + 1):
        pred = product_predictors[k].predictions["p_products_1"].iloc[i]
        pred = ut.canonicalize_smiles(pred)
        tgt = test["Product"].iloc[i]
        tgt_set = {i for i in tgt.split('.')}
        pred_prod = max(pred.split('.'), key=len)
        if pred_prod in tgt_set:
            hits += 1
    print(k, hits)

In [ ]:
for k in product_predictors:
    print(k)
    prod_topn_exact_match_acc = pd.concat(
        (
            test["Product"], 
            product_predictors[k].predictions
        )
        , axis=1).apply(lambda x: match_accuracy(x, 'exact'), axis=1)

    prod_topn_exact_match_acc = pd.DataFrame(prod_topn_exact_match_acc.to_list())
    prod_topn_exact_match_acc.columns = [f"top_{i + 1}_exact" for i in range(5)]

    print("Top-N exact match accuracy")
    for c in prod_topn_exact_match_acc.columns:
        print(c, prod_topn_exact_match_acc[c].sum())